In [24]:
import polars as pl
from datetime import datetime
import numpy as np
import time
from scipy.integrate import simps
import config

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [25]:
def remove_percentage(x): return int(x.split('%')[0])
def remove_MiB(x): return int(x.split('MiB')[0])
def remove_W(x): return float(x.split('W')[0])


def convert_to_timestamp(x): return int(datetime.strptime(
    x + " +0800", '%Y/%m/%d %H:%M:%S.%f %z').timestamp() * 1000)


In [67]:
def process(name: str):
    index_df = pl.read_csv(
        "index_nvidia_{}.txt".format(name))
    metrics_df = pl.read_csv("nvidiadata_{}.csv".format(name))
    metrics_df = metrics_df.with_columns([
        pl.col('utilizationgpu').apply(remove_percentage),
        pl.col('utilizationmemory').apply(remove_percentage),
        pl.col('memoryfree').apply(remove_MiB),
        pl.col('memoryused').apply(remove_MiB),
        pl.col('powerdraw').apply(remove_W),
        pl.col('timestamp').apply(convert_to_timestamp)
    ])
    metrics_df = metrics_df.drop('temperaturememory').drop(
        'pcielinkgencurrent').drop('index')
    data = []
    for i in range(index_df.shape[0]):

        model_name = index_df[i]['model'][0]
        batch_size = index_df[i]['batch_size'][0]
        start_time = index_df[i]['start_time'][0]
        end_time = index_df[i]['end_time'][0]
        duration = index_df[i]['duration'][0]

        filted_data = metrics_df.filter(
            pl.col('timestamp').is_between(
                start_time / 1000000, end_time / 1000000)
        ).to_numpy()[:, 1:]
        
        if filted_data.shape[0] >= 1024:
            length = 1024
            filted_data = filted_data[:1024]

        else:
            length = filted_data.shape[0]
            zero_padding = np.zeros((1024 - length, filted_data.shape[1]), dtype=np.float32)
            filted_data = np.concatenate((filted_data, zero_padding), axis=0)

        data.append(
            {
                'model': model_name,
                'batch_size': batch_size,
                'duration': duration,
                'data': filted_data,
                'length': length
            }
        )
        # data: 
        #       0 utilizationgpu,
        #       1 utilizationmemory,
        #       2 memoryfree,
        #       3 memoryused,
        #       4 temperaturegpu,
        #       5 powerdraw
    return data


In [82]:
data_transfer = []

all_data = {}

for x in ['A40', 'GTX_1080', 'RTX_2080', 'TITANX', 'TITANXp', 'V100']:
    all_data[x] = process(x)
    

In [83]:
data_label = {}

for x in ['A40', 'GTX_1080', 'RTX_2080', 'TITANX', 'TITANXp', 'V100']:
    data_label[x] = []
    for d in all_data[x]:
        i = 0
        while d['model'] + '_' + str(i) in data_label[x]:
            i += 1
        data_label[x].append(
            d['model'] + '_' + str(i)
        )

In [84]:
for x in ['A40', 'GTX_1080', 'RTX_2080', 'TITANX', 'TITANXp', 'V100']:
    for y in ['A40', 'GTX_1080', 'RTX_2080', 'TITANX', 'TITANXp', 'V100']:
        for data_label_x in data_label[x]:
            for data_label_y in data_label[y]:
                if data_label_x.split('_')[0] == data_label_y.split('_')[0]:
                    data_transfer.append(
                        {
                            'from': x,
                            'to': y,
                            'data_label': data_label_x.split('_')[0],
                            'from_batch_size': all_data[x][data_label[x].index(data_label_x)]['batch_size'],
                            'to_batch_size': all_data[y][data_label[y].index(data_label_y)]['batch_size'],
                            'from_duration': all_data[x][data_label[x].index(data_label_x)]['duration'],
                            'to_duration': all_data[y][data_label[y].index(data_label_y)]['duration'],
                            'from_metrics': all_data[x][data_label[x].index(data_label_x)]['data'],
                            'from_length': all_data[x][data_label[x].index(data_label_x)]['length']
                        }
                    )
                
                

In [87]:
len(data_transfer)

3021027

In [88]:
data_transfer[0]

{'from': 'A40',
 'to': 'A40',
 'data_label': 'hf',
 'from_batch_size': 1,
 'to_batch_size': 1,
 'from_duration': 11576.40234375,
 'to_duration': 11576.40234375,
 'from_metrics': array([[1.0000e+02, 9.0000e+01, 4.0077e+04, 5.5570e+03, 3.2000e+01,
         1.5479e+02],
        [9.9000e+01, 9.1000e+01, 4.0077e+04, 5.5570e+03, 3.3000e+01,
         1.6948e+02],
        [9.9000e+01, 9.1000e+01, 4.0077e+04, 5.5570e+03, 3.3000e+01,
         1.8035e+02],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00]]),
 'from_length': 110}

In [89]:
np.save("transfer_data.npy", data_transfer)

In [49]:
data = np.load('sequence_data.npy', allow_pickle=True)

In [50]:
t = set()
for d in data:
    if d['data'].shape[0] != 1024:
        print(d['data'].shape[0], d['length'])

In [30]:
modelnamelist = sorted(['timm_nfnet', 'resnet50_quantized_qat', 'shufflenet_v2_x1_0', 'densenet121', 'detectron2_maskrcnn', 'vgg16', 'hf_Bart', 'yolov3', 'hf_DistilBert', 'resnet18', 'hf_GPT2', 'hf_Albert', 'mobilenet_v3_large', 'maml_omniglot', 'timm_vision_transformer', 'maml', 'vision_maskrcnn', 'alexnet', 'nvidia_deeprecommender', 'LearningToPaint', 'resnext50_32x4d', 'hf_Bert', 'drq', 'timm_vovnet', 'pytorch_unet', 'hf_T5', 'mobilenet_v2', 'timm_regnet', 'squeezenet1_1', 'pytorch_CycleGAN_and_pix2pix', 'hf_Reformer', 'dcgan', 'Super_SloMo', 'mobilenet_v2_quantized_qat', 'hf_Longformer', 'resnet50', 'timm_efficientnet', 'attention_is_all_you_need_pytorch', 'BERT_pytorch', 'mnasnet1_0', 'fastNLP_Bert', 'hf_BigBird'])
model_name_to_num = {modelnamelist.index(model): model for model in modelnamelist}
model_num_to_name = {model: modelnamelist.index(model) for model in modelnamelist}

In [32]:
data[0].keys()

dict_keys(['model', 'batch_size', 'duration', 'data', 'length'])

In [3]:
import ShanghaitechData

ds = ShanghaitechData.ShanghaitechClusterDataset()

import torch
from torch.utils.data import DataLoader
dl = DataLoader(ds, batch_size=20, shuffle=True)


In [5]:
for d in dl:
    (from_device_config, 
    to_device_config, 
    from_batch_size, 
    to_batch_size, 
    from_metrics, 
    from_length,
    duration_rate) = d
    break

In [6]:
from_metrics.shape

torch.Size([20, 1024, 6])

In [8]:
from_metrics

tensor([[[2.4000e+01, 5.0000e+00, 4.3511e+04, 2.1230e+03, 4.5000e+01,
          8.8490e+01],
         [2.4000e+01, 5.0000e+00, 4.3511e+04, 2.1230e+03, 4.5000e+01,
          9.0320e+01],
         [3.0000e+01, 7.0000e+00, 4.3511e+04, 2.1230e+03, 4.5000e+01,
          9.2350e+01],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00]],

        [[8.8000e+01, 5.6000e+01, 1.0979e+04, 1.2170e+03, 8.2000e+01,
          2.4525e+02],
         [8.9000e+01, 5.9000e+01, 1.0979e+04, 1.2170e+03, 8.2000e+01,
          2.5639e+02],
         [9.4000e+01, 6.1000e+01, 1.0979e+04, 1.2170e+03, 8.2000e+01,
          2.5089e+02],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000

In [10]:
gru = torch.nn.GRU(input_size=6, hidden_size=20, dropout=0.5, batch_first=True)
result = gru(from_metrics.float())

In [15]:
result[1]

tensor([[[ 0.3028,  0.2551,  0.2603,  0.2040,  0.1526,  0.0818,  0.1320,
          -0.0558, -0.1966, -0.1944, -0.1103,  0.3012,  0.0171, -0.0691,
           0.1160,  0.0656, -0.2441,  0.1702,  0.0156, -0.0139],
         [ 0.3028,  0.2551,  0.2603,  0.2040,  0.1526,  0.0818,  0.1320,
          -0.0558, -0.1966, -0.1944, -0.1103,  0.3012,  0.0171, -0.0691,
           0.1160,  0.0656, -0.2441,  0.1702,  0.0156, -0.0139],
         [ 0.3028,  0.2551,  0.2603,  0.2040,  0.1526,  0.0818,  0.1320,
          -0.0558, -0.1966, -0.1944, -0.1103,  0.3012,  0.0171, -0.0691,
           0.1160,  0.0656, -0.2441,  0.1702,  0.0156, -0.0139],
         [ 0.3028,  0.2551,  0.2603,  0.2040,  0.1526,  0.0818,  0.1320,
          -0.0558, -0.1966, -0.1944, -0.1103,  0.3012,  0.0171, -0.0691,
           0.1160,  0.0656, -0.2441,  0.1702,  0.0156, -0.0139],
         [ 0.3028,  0.2551,  0.2603,  0.2040,  0.1526,  0.0818,  0.1320,
          -0.0558, -0.1966, -0.1944, -0.1103,  0.3012,  0.0171, -0.0691,
          